# 📊 **Re-Implementation of "Predicting Food Crises Using News Streams"**

---

#### 🔍 **Objective**

This notebook aims to **reproduce and analyze** the methodology presented in the paper:

📄 **Paper:** [Predicting food crises using news streams](https://www.science.org/doi/10.1126/sciadv.abm3449)  
📊 **Dataset:** [Harvard Dataverse Repository](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/CJDWUW)  
📜 **Original Code & Methods:** [GitHub - Regression Modeling (Step 5)](https://github.com/philippzi98/food_insecurity_predictions_nlp/blob/main/Step%205%20-%20Regression%20Modelling/README.md)

---

#### 🛠 **Methodology**

This implementation follows the **key steps** outlined in the paper to predict **food insecurity crises** using a combination of:
1️⃣ **Traditional Risk Factors** (conflict, climate, food prices, etc.)  
2️⃣ **News-Based Indicators** (text feature frequencies from news articles)  
3️⃣ **Lagging & Aggregation** (temporal dependencies at district, province, and country levels)  
4️⃣ **Machine Learning Models** (Random Forest, OLS, Lasso)

---

#### 🔗 **Reference Materials**

📄 **Supplementary Material:** Available in `supplemental_material_from_paper.pdf`  
📊 **Datasets Used:**

- `time_series_with_causes_zscore_full.csv` (Main dataset with time-series features)
- `famine-country-province-district-years-CS.csv` (Food insecurity classification)
- `matching_districts.csv` (Geographical standardization)


# 📚🔧 Import Libraries

In this notebook, we will use uv to manage our Python environment and packages efficiently. uv is a modern and fast package manager that simplifies virtual environment creation, and dependency installation. We will create a virtual environment, install necessary libraries, and ensure our environment stays consistent across different setups.


In [308]:
## Uncoment the below cell to install `uv` if you have not already. You can also install it trhiugh `pip` by running `!pip install uv` but this will be within your current python environment and not globally.

# !curl -LsSf https://astral.sh/uv/install.sh | sh
# !uv venv world-bank
# !source world-bank/bin/activate

In [309]:
# !uv pip install -r requirements.txt

In [310]:
import pandas as pd
import numpy as np
from IPython.display import display, Image
import os
import gdown
import zipfile
from fuzzywuzzy import fuzz
import math

In [311]:
url = "https://drive.google.com/uc?id=1YoQ1hz9RlaLr2xW3KoKCfJPyyO2PErym"
output = "data.zip"

if not os.path.exists("./data"):
    gdown.download(url, output, quiet=False) 
    zipfile.ZipFile('data.zip', 'r').extractall()
else:
    print("You already have the data downloaded and extracted")

You already have the data downloaded and extracted


## 📂 Load and Clean Data

**Understanding the Time-Series Dataset & Column Selection**

This dataset contains **district-level time-series data** on food insecurity risk factors, including:

- **📅 Temporal Information:** `year`, `month`, `year_month`
- **📍 Geographical Identifiers:** `admin_code`, `admin_name`, `province`, `country`
- **🌍 Traditional Risk Factors:** Climate (`rain_mean`, `ndvi_mean`), conflict (`acled_count`), food prices (`p_staple_food`)
- **📰 News-Based Indicators:** Proportions of news articles mentioning crisis-related keywords (`conflict_0`, `famine_0`, etc.)
- **📉 Food Insecurity Label:** `fews_ipc` (Integrated Phase Classification)

🔥 **Columns We Will Drop & Why**
✔ **Redundant Aggregations:** `_1`, `_2` columns (province & country-level values) since we will recompute aggregations from scratch anyways.  
✔ **Unnamed/Index Columns:** `Unnamed: 0` as it is unnecessary. It is just a duplicate of default index.
✔ **Unnecessary Identifiers:** If `admin_code` and `admin_name`, after matching these to `matching_districts.csv`, we can drop them.

---

> ⚠️ **NOTE:**  
> For a detailed explanation of the dataset and features, refer to the [`explore_time_series.ipynb`](./explore_time_series.ipynb) notebook.


In [312]:
time_series = pd.read_csv('./data/time_series_with_causes_zscore_full.csv', nrows=30)
admins = pd.read_csv('./data/famine-country-province-district-years-CS.csv')
valid_matching = pd.read_csv('./data/matching_districts.csv')

In [313]:
# sorted(time_series.columns.values)

In [314]:
time_series.head(5)

,Unnamed: 0,index,country,admin_code,admin_name,centx,centy,year_month,year,month,...,carbon_2,mayhem_0,mayhem_1,mayhem_2,dehydrated_0,dehydrated_1,dehydrated_2,mismanagement_0,mismanagement_1,mismanagement_2
0,0,30,Afghanistan,202,Kandahar,65.709343,31.043618,2009_07,2009,7,...,1.053000,0.667000,-0.171000,-0.833000,0.173667,0.168000,1.284667,-0.073000,-0.427667,0.668333
1,1,33,Afghanistan,202,Kandahar,65.709343,31.043618,2009_10,2009,10,...,-0.660812,-0.636580,-0.520247,-0.782913,-0.671587,-0.612254,-0.926921,-0.510467,-0.625133,-0.452467
2,2,36,Afghanistan,202,Kandahar,65.709343,31.043618,2010_01,2010,1,...,-0.134333,1.447667,-0.844333,0.778667,-0.676000,-0.689667,0.293333,0.530333,-0.471333,0.955333
3,3,39,Afghanistan,202,Kandahar,65.709343,31.043618,2010_04,2010,4,...,-0.326927,-0.594877,0.164790,-0.905210,-0.620540,0.165794,0.045794,-1.011600,-0.810600,-0.205600
4,4,42,Afghanistan,202,Kandahar,65.709343,31.043618,2010_07,2010,7,...,-1.085146,-0.709913,-0.867913,-0.770247,-0.787921,-0.974587,-0.946921,-0.611133,-0.709800,-0.622800


In [315]:
t_variant_traditional_factors = ['ndvi_mean', 'ndvi_anom', 'rain_mean', 'rain_anom', 'et_mean', 'et_anom', 
                                    'acled_count', 'acled_fatalities', 'p_staple_food']
t_invariant_traditional_factors = ['area', 'cropland_pct', 'pop', 'ruggedness_mean', 'pasture_pct']
news_factors = [name for name in time_series.columns.values if '_0' in name]

In [316]:
news_factors[0]

'land seizures_0'

In [317]:
print("Columns count BEFORE dropping: ", len(time_series.columns.values))

Columns count BEFORE dropping:  532


In [318]:
cols_to_drop = ["Unnamed: 0", "centx", "centy", 'change_fews', 'fews_ha', 'fews_proj_med', 'fews_proj_med_ha', 'fews_proj_near_ha'] + [col for col in time_series.columns if col.endswith(('_1', '_2', '_3'))]
time_series.drop(columns=cols_to_drop, inplace=True)

In [319]:
potential_extra_cols = set(time_series.columns.values) - set(t_variant_traditional_factors) - set(t_invariant_traditional_factors) - set(news_factors)
potential_extra_cols = [col for col in potential_extra_cols if not col.endswith(('_1', '_2', '_3'))]
print("Potential extra columns", potential_extra_cols)

Potential extra columns ['country', 'admin_name', 'fews_ipc', 'fews_proj_near', 'year', 'year_month', 'admin_code', 'month', 'index']


In [320]:
print("Columns count after dropping: ", len(time_series.columns.values))

Columns count after dropping:  190


### 🌍 Admin Level Mapping: Standardizing Geographical Identifiers

In this section, we will **map and standardize** the `admin_code` and `admin_name` fields to their corresponding **district, province, and country names**. This step is **crucial** for ensuring **consistency** across different datasets and enabling **accurate aggregations** at multiple administrative levels.

🛠 **Why is Admin Level Mapping Important?**
✅ Different datasets may use **slightly different spellings or formats** for district names.  
✅ Some district names might be **missing or misspelled**, requiring standardization.  
✅ We need to **match and align** district names across various sources before aggregating at **province and country levels**.  
✅ Proper mapping allows us to **merge datasets correctly** without losing information.  

📌 **Steps in Admin Mapping**
1️⃣ **Load the `matching_districts.csv` file**, which provides the mapping between different district name variations.  
2️⃣ **Identify missing or unmatched `admin_name` values** and find their closest matches using fuzzy matching techniques.  
3️⃣ **Ensure that each `admin_code` uniquely maps to one `district`, `province`, and `country`.**  
4️⃣ **Replace inconsistent names** in the dataset with their standardized versions.  
5️⃣ **Aggregate data at the `province` and `country` levels** after ensuring all districts are correctly mapped.  


In [321]:
len(admins.country.unique())

39

In [322]:
admins.columns.values

array(['Unnamed: 0', 'country', 'district', 'year', 'month', 'CS',
       'province'], dtype=object)

In [323]:
admin_names = time_series['admin_name'].unique()
districts = admins['district'].unique()
provinces = admins['province'].unique()
countries = admins['country'].unique()

In [324]:
print (len(admin_names), len(districts), len(provinces), len(countries))
print (len(set(admin_names).difference(districts)))
missing_admin_names = set(admin_names).difference(districts)
print (len(missing_admin_names.difference(provinces)))
missing_admin_names = missing_admin_names.difference(provinces)

1 4113 474 39
0
0


### Fuzzy String Matching for Missing Names

The function uses **fuzzy string matching** to find the best approximate matches for missing administrative names (e.g., districts and provinces). 

- Finds the **best matching district/province** for each missing name.
- Uses **fuzzy string matching** to calculate the similarity between missing names and known names.
- Returns a dictionary that maps each missing name to its closest match.


In [325]:
def find_matching(missing, names):
    matching_districts = {}
    for m in missing:
        max_overlap = 0
        nearest_d = None
        for d in names:
            d = str(d)
            dist = fuzz.partial_ratio(m, d)
            if dist > max_overlap:
                max_overlap = dist
                nearest_d = d
        matching_districts[m] = nearest_d
    return matching_districts


matching = find_matching(missing_admin_names, districts)
matching_p = find_matching(missing_admin_names, provinces)

# manually verify matching and update
for k in matching.keys():
    print (k, matching[k], matching_p[k])


### Encoding Decoding

`to_ascii_escaped(s)`: Converts a Unicode string to an ASCII-safe representation using **unicode-escape**.

`from_ascii_escaped(escaped)`: Converts the escaped ASCII string back into its original Unicode form.

In [326]:
def to_ascii_escaped(s):
    """
    Convert a Unicode string to an ASCII-safe string using unicode-escape.
    This will replace non-ASCII characters with their escape sequences.
    """
    if isinstance(s, bytes):
        s = s.decode('utf-8')
    # Using 'unicode-escape' encoding produces a bytes object,
    # then decode it to get an ASCII string.
    return s.encode('unicode-escape').decode('ascii')

def from_ascii_escaped(escaped):
    """
    Convert the ASCII-escaped string back to the original Unicode string.
    """
    # Encode the ASCII string to bytes, then decode using 'unicode-escape'
    return escaped.encode('ascii').decode('unicode-escape')


### Finding the Province for a Given District or Province

`find_province(x)`, finds the **province** corresponding to a given administrative name. It accounts for:
- **Direct Lookups** (Exact match in known district/province lists)
- **Fuzzy Matching** (Using ASCII-safe transformation for inconsistent text encoding)
- **Validation Against a Predefined Mapping (`valid_matching`)**

In [327]:
# Define matched globally
matched = valid_matching['missing'].unique()

def to_ascii_escaped(s):
    """
    Convert a Unicode string to an ASCII-safe string using unicode-escape.
    This will replace non-ASCII characters with their escape sequences.
    """
    if isinstance(s, bytes):
        s = s.decode('utf-8')
    return s.encode('unicode-escape').decode('ascii')

def find_province(x):
    try:
        # Ensure x is a Unicode string.
        if isinstance(x, bytes):
            x = x.decode('utf-8')
        
        # Direct lookup in districts or provinces.
        if x in districts:
            return admins[admins['district'] == x]['province'].values[0]
        elif x in provinces:
            return x

        # Convert x to an ASCII-escaped version.
        escaped_x = to_ascii_escaped(x)
        
        # Check if the escaped version is in matched.
        if escaped_x in matched:
            v = valid_matching[valid_matching['missing'] == escaped_x]
            if v['match'].values[0] == 'district':
                x2 = v['district'].values[0]
                return admins[admins['district'] == x2]['province'].values[0]
            elif v['match'].values[0] == 'province':
                return v['province'].values[0]
        
        # If no conditions are met, raise an exception.
        raise Exception("No matching province found")
    except Exception as e:
        raise Exception("Province not found for: {} ({})".format(x, e))


### Handling Admin Names with Accented Characters and Mapping to Provinces

Maps `admin_names` to provinces using the `find_province(a)` function.  
If a **direct lookup fails**, it tries to handle cases where the **admin name contains accented characters** (`é`, `è`, `ô`) ->  (encoding decoding issues resolved through directly replacing these with 'e' or 'o', leads to finding a valid match). 

In [328]:
admin_to_province = {}
for a in admin_names:
    try:
        admin_to_province[a] = find_province(a)
    except Exception as e:
        # Print the admin name that caused an error
        print("Error with:", a)
        # Check if a contains accented characters "é" or "è"
        if 'é' in a or 'è' in a or 'ô' in a:
            a_modified = a.replace('é', 'e').replace('è', 'e').replace('ô', 'o')
            # Check if the modified name is in districts
            if a_modified in districts:
                # Use the modified name to look up the province from admins
                try:
                    province = admins[admins['district'] == a_modified]['province'].values[0]
                    admin_to_province[a] = province
                    print(f"Replaced '{a}' with '{a_modified}', found province: {province}")
                except Exception as ex:
                    print(f"Modified name '{a_modified}' not found in admins: {ex}")
            else:
                print(f"Modified name '{a_modified}' not in districts.")
        else:
            print(f"No accented e found in '{a}'.")


### Mapping Administrative Names to Provinces in time_series

Maps `admin_name` to their respective **provinces** using a precomputed dictionary - >`admin_to_province` in `time_series`.


In [329]:
time_series['province'] = time_series['admin_name'].apply(
    lambda x: admin_to_province[x] if x in admin_to_province else admin_to_province.get(x.replace('ô', 'o'))
)


In [330]:
# time_series[["admin_name", "province"]]

# ⏳ Time Lagging & Feature Engineering

#### 📅 **Why Use Lagging?**

To predict food insecurity **for a given quarter**, we use:

- **6 months of historical values** for traditional & news-based features.
- **Province & country-level aggregations** to capture broader shocks.
- **6 quarters of lagged IPC phase values** to model temporal dependencies.

#### ⚡ **Optimized Lagging Approach**

To improve computational efficiency, we:
✔ Use `groupby()` for **fast province & country-level aggregations**.  
✔ Merge lagged data via `merge()` instead of slow `.apply()`.  
✔ Only keep **past data** to ensure no data leakage.


In [331]:
def add_time_lagged(features, start=3, end=9, diff=1, agg=True, time_series=time_series):
    
    # Determine suffixes based on aggregation flag
    levels = ['', '_province', '_country'] if agg else ['']
    
    # Create a copy of the dataframe to work with
    # This helps with defragmentation and prevents modifying the original during processing
    working_df = time_series.copy()
    
    # Create a mapping from (admin_code, year, month) to row index for fast lookups
    # We'll use this for efficient merging of lagged data
    # working_df['total_months'] = working_df['year'] * 12 + working_df['month'] - 1
    # working_df['total_months'] = working_df['year'] 
    
    
    # Prepare a list to collect all new dataframes
    new_columns_df = []
    
    # Track new column names to add
    new_column_names = []
    
    # Process each feature and lag combination
    for suffix in levels:
        for f in features:
            f_s = f + suffix
            
            # # Skip if the feature doesn't exist
            # if f_s not in working_df.columns:
            #     continue
                
            for t in range(start, end, diff):
                col_name = f'{f_s}_{t}'
                
                # Skip if column already exists
                if col_name in time_series.columns:
                    continue
                
                new_column_names.append(col_name)
                
                # Create a dataframe with the lagged feature
                lagged_df = working_df[['admin_code', f_s]].copy()
                
                # # Calculate the target months to look back to
                # lagged_df['target_months'] = lagged_df['total_months'] - t
                
                # # Convert target months back to year and month
                # lagged_df['l_year'] = lagged_df['target_months'] // 12
                # lagged_df['l_month'] = (lagged_df['target_months'] % 12) + 1
                
                year = working_df['year']
                month = working_df['month']
                l_month = ((month-1-t)%12)+1
                l_year = year
                # if month-t<=0:
                #     l_year = year-1
                l_year = np.where(month - t <= 0, year - 1, year)
                lagged_df['l_year'] = l_year
                lagged_df['l_month'] = l_month
                
                # Create a reference key for merging
                lagged_df['ref_key'] = lagged_df['admin_code'].astype(str) + '_' + \
                                       lagged_df['l_year'].astype(str) + '_' + \
                                       lagged_df['l_month'].astype(str)
                
                # Create the same key in our original dataframe for merging
                working_df['temp_key'] = working_df['admin_code'].astype(str) + '_' + \
                                        working_df['year'].astype(str) + '_' + \
                                        working_df['month'].astype(str)
                
                # Map from reference key to the feature value
                ref_map = dict(zip(
                    working_df['temp_key'],
                    working_df[f_s]
                ))
                
                # Apply the mapping to get lagged values - use the original value as fallback
                lagged_values = lagged_df['ref_key'].map(ref_map).fillna(working_df[f_s])
                
                # Create a single-column dataframe with the lagged values
                lag_result_df = pd.DataFrame({
                    col_name: lagged_values.values
                }, index=working_df.index)
                
                new_columns_df.append(lag_result_df)
    
    # Clean up temp columns
    if 'temp_key' in working_df.columns:
        working_df.drop('temp_key', axis=1, inplace=True)
    
    # If we have new columns to add, concatenate them all at once
    if new_columns_df:
        # Combine all new columns into a single dataframe
        all_new_columns = pd.concat(new_columns_df, axis=1)
        
        # Add the new columns to the original dataframe
        time_series = pd.concat([time_series, all_new_columns], axis=1)
        
    return time_series

In [332]:
# def get_lagged(x, f, t, ts_dict):
#     admin_code = x['admin_code']
#     year, month = x['year'], x['month']

#     # Compute lagged year and month
#     l_month = ((month - 1 - t) % 12) + 1
#     l_year = year - 1 if month - t <= 0 else year
#     lagged_year_month = f"{l_year}_{l_month}"

#     # Retrieve pre-filtered DataFrame
#     ts = ts_dict.get(admin_code)

#     # Ensure ts is a dictionary and extract only the requested feature
#     if ts is not None and lagged_year_month in ts:
#         lagged_values = ts[lagged_year_month]  # This might be a dictionary
#         if isinstance(lagged_values, dict):
#             return lagged_values.get(f, x[f])  # Extract only `f`
#         return lagged_values  # Directly return value if not a dict
    
#     print(f"Ts is none")

#     return x[f]  # Fallback to original value


# def add_time_lagged(features, start=3, end=9, diff=1, agg=True):
#     levels = ['', '_province', '_country'] if agg else ['']

#     # Convert 'year_month' to index and precompute admin_code groupings
#     ts_dict = {ac: df.set_index('year_month')[features].to_dict(orient="index")
#                for ac, df in time_series.groupby('admin_code')}

#     for suffix in levels:
#         for f in features:
#             f_s = f + suffix
#             for t in range(start, end, diff):
#                 lagged_col = f"{f_s}_{t}"

#                 if lagged_col in time_series.columns:
#                     continue

#                 # Use list comprehension for better performance
#                 time_series[lagged_col] = [get_lagged(row, f_s, t, ts_dict) for _, row in time_series.iterrows()]

#     return time_series  # Return modified DataFrame


# Province & Country-Level Aggregation

This function aggregates feature values at the province and country levels to capture regional trends, aiding in food insecurity prediction. The process includes:

- **Grouping by year_month and level:** Data is grouped by year_month and the specified level (province or country) to calculate the mean of features, reflecting regional trends over time.

- **Applying transformations efficiently:** Instead of merging aggregated data, `transform("mean")` is used to directly assign the computed mean to each row, avoiding unnecessary joins and improving performance.  

#### ⚡ **Efficiency Gains**

- **Fast Aggregation**: Uses `groupby()` for efficient aggregation.
- **Avoids Costly Joins**: Eliminates the need for `merge()` by using `transform()` instead, reducing computational overhead.  
- **Memory Efficiency**: Converts the `level` column to a categorical type to reduce memory usage.

This approach ensures faster processing while maintaining the quality of aggregated features.


In [333]:
def add_agg_factors(features, level='province'):
    global time_series  

    # Convert 'level' column to categorical for performance
    time_series[level] = time_series[level].astype('category')
    
    # Compute grouped mean values for the given features
    # TODO : Explain these arguments
    grouped_df = time_series.groupby(['year_month', level], observed=True, sort=False)[features].transform("mean")

    # Rename columns to include level
    grouped_df = grouped_df.rename(columns={f: f"{f}_{level}" for f in features})

    # Use pd.concat() to add all columns at once, avoiding fragmentation
    time_series = pd.concat([time_series, grouped_df], axis=1)

    return time_series


In [334]:
t = add_agg_factors(news_factors)

In [335]:
t = add_agg_factors(news_factors, level='country')
time_series.head(10)

,index,country,admin_code,admin_name,year_month,year,month,fews_ipc,fews_proj_near,ndvi_mean,...,gastrointestinal_0_country,terrorist_0_country,warlord_0_country,d'etat_0_country,overthrow_0_country,convoys_0_country,carbon_0_country,mayhem_0_country,dehydrated_0_country,mismanagement_0_country
0,30,Afghanistan,202,Kandahar,2009_07,2009,7,1.0,NaN,0.106035,...,-0.192000,-0.284333,-0.668667,0.647333,-0.891333,0.112667,1.265333,0.667000,0.173667,-0.073000
1,33,Afghanistan,202,Kandahar,2009_10,2009,10,1.0,NaN,0.103009,...,-0.545727,-1.037016,-0.811291,-0.850261,-0.948892,-0.728972,-0.765146,-0.636580,-0.671587,-0.510467
2,36,Afghanistan,202,Kandahar,2010_01,2010,1,2.0,NaN,0.109600,...,1.506333,0.455000,1.595667,0.571667,0.279000,-0.868333,0.058333,1.447667,-0.676000,0.530333
3,39,Afghanistan,202,Kandahar,2010_04,2010,4,2.0,NaN,0.111599,...,-0.793970,-0.722159,-0.130521,0.047630,0.362613,0.480986,0.026073,-0.594877,-0.620540,-1.011600
4,42,Afghanistan,202,Kandahar,2010_07,2010,7,1.0,NaN,0.096943,...,-0.509394,-0.694350,-1.215958,-0.865261,-1.119225,-1.060638,-0.673479,-0.709913,-0.787921,-0.611133
5,45,Afghanistan,202,Kandahar,2010_10,2010,10,2.0,NaN,0.095377,...,-0.691000,1.168667,-0.279333,-0.296667,1.651333,-0.356000,1.156667,1.202667,0.446667,0.696667
6,48,Afghanistan,202,Kandahar,2011_01,2011,1,2.0,NaN,0.092620,...,-0.916000,0.334333,-0.847000,1.270000,0.744333,1.328667,-0.705000,0.738000,0.358000,0.357667
7,51,Afghanistan,202,Kandahar,2011_04,2011,4,2.0,2.0,0.131462,...,0.098364,-0.792159,-0.083854,-0.229370,0.050279,0.181986,-0.413594,0.278123,-0.235540,-0.817600
8,54,Afghanistan,202,Kandahar,2011_07,2011,7,1.0,1.0,0.106885,...,-0.820667,-0.696000,-0.588667,0.714000,1.049667,-0.590000,-0.759667,-0.057000,0.993333,1.217667
9,57,Afghanistan,202,Kandahar,2011_10,2011,10,1.0,1.0,0.103268,...,0.339000,-0.851667,0.421000,1.312000,0.010667,1.056333,1.055333,0.844000,0.652000,-0.213000


In [336]:
t = add_agg_factors(t_variant_traditional_factors, level='province')

In [337]:
t = add_agg_factors(t_variant_traditional_factors, level='country')
t = add_agg_factors(t_invariant_traditional_factors, level='province')
t = add_agg_factors(t_invariant_traditional_factors, level='country')

In [338]:
# time_series.to_csv('ours_agg_province_features_full.csv')

# Add time lagged features


In [339]:
time_series = add_time_lagged(t_variant_traditional_factors, time_series=time_series)

In [340]:
# time_series = add_time_lagged(news_factors, time_series=time_series)

In [341]:
# time_series = add_time_lagged(['fews_ipc'], end=21, diff=3, agg=False, time_series=time_series)

In [342]:
# time_series = add_time_lagged(['fews_proj_near'], start=3, end=4, diff=1, agg=False, time_series=time_series)

In [343]:
def diebold_mariano(preds, labels):
    sq_error = [(p-l)**2 for p,l in zip(preds, labels)]
    mean = np.mean(sq_error)
    n = len(preds)
    gammas = {}
    m = max(n,int(math.ceil(np.cbrt(n))+2))
    for k in range(m):
        gammas[k] = 0
        for i in range(k+1, n):
            gammas[k] += (sq_error[i] - mean)*(sq_error[i-k] - mean)
        gammas[k] = gammas[k]/n
    sum_gamma = gammas[0]
    for k in range(1, m):
        sum_gamma += 2*gammas[k]
    return np.sqrt(sum_gamma/n)

In [344]:
time_series.columns.shape

(715,)

In [345]:
time_series.to_csv("our_results_tvariant_30.csv")

In [346]:
time_series.head()

,index,country,admin_code,admin_name,year_month,year,month,fews_ipc,fews_proj_near,ndvi_mean,...,acled_fatalities_country_5,acled_fatalities_country_6,acled_fatalities_country_7,acled_fatalities_country_8,p_staple_food_country_3,p_staple_food_country_4,p_staple_food_country_5,p_staple_food_country_6,p_staple_food_country_7,p_staple_food_country_8
0,30,Afghanistan,202,Kandahar,2009_07,2009,7,1.0,NaN,0.106035,...,0.0,0.0,0.0,0.0,1.065669,1.065669,1.065669,1.065669,1.065669,1.065669
1,33,Afghanistan,202,Kandahar,2009_10,2009,10,1.0,NaN,0.103009,...,0.0,0.0,0.0,0.0,1.065669,1.100531,1.100531,1.100531,1.100531,1.100531
2,36,Afghanistan,202,Kandahar,2010_01,2010,1,2.0,NaN,0.109600,...,0.0,0.0,0.0,0.0,1.100531,0.988390,0.988390,1.065669,0.988390,0.988390
3,39,Afghanistan,202,Kandahar,2010_04,2010,4,2.0,NaN,0.111599,...,0.0,0.0,0.0,0.0,0.988390,0.992492,0.992492,1.100531,0.992492,0.992492
4,42,Afghanistan,202,Kandahar,2010_07,2010,7,1.0,NaN,0.096943,...,0.0,0.0,0.0,0.0,0.992492,1.024889,1.024889,0.988390,1.024889,1.024889


In [347]:
# find columns that contain a particular substring

# list(filter(lambda x: 'co' in x, time_series.columns))

In [348]:
# find columns that begin with a particular substring
list(filter(lambda x: x.startswith("coun"), time_series.columns))


['country']

# Generate and save data


In [349]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import root_mean_squared_error, precision_recall_curve, auc

train_splits = [((2009,7), (2010,4)), ((2009,7), (2011,1)), ((2009,7), (2011,10)), 
                ((2009,7), (2012,7)), ((2009,7), (2013,7)), ((2009,7), (2014,1)), 
                ((2009,7), (2015,1)), ((2009,7), (2015,10)), ((2009,7), (2016,10)), 
                ((2009,7), (2017,2))]

dev_splits = [((2010,4), (2010, 7)), ((2011,1), (2011, 7)), ((2011,10), (2012, 7)), 
              ((2012,7), (2013, 7)), ((2013,4), (2014, 7)), ((2014,1), (2015, 7)), 
              ((2015,1), (2016, 7)), ((2015,10), (2017, 7)), ((2016,10), (2018, 7)), 
              ((2017,2), (2019, 2))]

test_splits = [((2010,7), (2011, 7)), ((2011,7), (2012, 7)), ((2012,7), (2013, 7)), 
               ((2013,7), (2014, 7)), ((2014,7), (2015, 7)), ((2015,7), (2016, 7)), 
               ((2016,7), (2017, 7)), ((2017,7), (2018, 7)), ((2018,7), (2019, 7)), 
               ((2019,2), (2020, 2))]

# just like them we will evaluate three dufferent models, Random Forest, OLS and Lasso. Random Forest is a tree-based model, OLS is a linear regression model and Lasso is a linear regression model with L1 regularization
models = {
    'RF': RandomForestRegressor(max_features='sqrt', n_estimators=100, min_samples_split=0.5, min_impurity_decrease=0.001, random_state=0),
    'OLS': LinearRegression(),
    'Lasso': Lasso(alpha=0.1)
}

def get_agg_lagged_features(factors):
    return [f"{f}_{t}" for f in factors for t in range(3, 9)] + \
           [f"{f}_province_{t}" for f in factors for t in range(3, 9)] + \
           [f"{f}_country_{t}" for f in factors for t in range(3, 9)]

features = {
    'traditional': time_series[['year', 'month'] + 
        [f"fews_ipc_{t}" for t in range(3, 21, 3)] + 
        get_agg_lagged_features(t_variant_traditional_factors) + 
        t_invariant_traditional_factors],
    
    'news': time_series[['year', 'month'] + 
        [f"fews_ipc_{t}" for t in range(3, 21, 3)] + 
        get_agg_lagged_features(news_factors)],
    
    'traditional+news': time_series[['year', 'month'] + 
        [f"fews_ipc_{t}" for t in range(3, 21, 3)] + 
        get_agg_lagged_features(t_variant_traditional_factors) + 
        t_invariant_traditional_factors + 
        get_agg_lagged_features(news_factors)]
}

labels_df = time_series[['fews_ipc', 'year', 'month']]

def get_time_split(df, start, end):
    return df[
        (((df['year'] > start[0])) | ((df['year'] == start[0]) & (df['month'] >= start[1]))) &
        (((df['year'] < end[0])) | ((df['year'] == end[0]) & (df['month'] <= end[1])))
    ]

thresholds = {
    'traditional': (2.236, 3.125), 'news': (1.907, 2.712), 'traditional+news': (2.105, 3.314),
} # (lowerbound, upperbound)

def train_and_evaluate(train, dev, test, f, D):
    results = []

    X_train = get_time_split(D, train[0], dev[1]).drop(columns=['year', 'month']).fillna(0).to_numpy() # not sure how okay it is to do fillna. When me and Bilal were running this we were getting the error that cannot run the model on NaN values. First we dropped na but this was causing the shape of the X_train to be different from the y_train. So we decided to fillna with 0. - aysha & bilal
    y_train = get_time_split(labels_df, train[0], dev[1]).drop(columns=['year', 'month']).to_numpy().ravel()
    
    X_test = get_time_split(D, test[0], test[1]).drop(columns=['year', 'month']).fillna(0).to_numpy()
    y_test = get_time_split(labels_df, test[0], test[1]).drop(columns=['year', 'month']).to_numpy().ravel()
    
    # convert y_test into binary classification (1 if inside threshold, else 0)
    lower, upper = thresholds[f]
    y_test_binary = np.where((y_test >= lower) & (y_test <= upper), 1, 0)

    for name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        rmse = root_mean_squared_error(y_test, preds)

        stderr = np.std(y_test - preds) / np.sqrt(len(y_test))
        upper_bound = np.sqrt(rmse**2 + 1.96 * stderr)
        lower_bound = np.sqrt(rmse**2 - 1.96 * stderr)

        precision, recall, _ = precision_recall_curve(y_test_binary, preds)
        aucpr = auc(recall, precision)

        results.append({
            'method': name, 'split': test, 'features': f, 
            'rmse': rmse, 'lower_bound': lower_bound, 'upper_bound': upper_bound,
            'aucpr': aucpr
        })

        print(f"Method: {name}, Split: {test}, Features: {f}, AUCPR: {aucpr:.4f}")
        print(f"Method: {name}, Split: {test}, Features: {f}, RMSE: {rmse:.4f} [{lower_bound:.4f}, {upper_bound:.4f}]")
        
        # completely removed the part where they were doing country-wise evaluation. Do not see point - aysha
    
    return results

# run in parallel on 4 cpu cores/decrease this if you do not want ur system to crash (speaking from experience)
all_results = Parallel(n_jobs=4)(
    delayed(train_and_evaluate)(train, dev, test, f, D) for train, dev, test in zip(train_splits, dev_splits, test_splits) for f, D in features.items()
)

fig_3a = pd.DataFrame([res for sublist in all_results for res in sublist])
fig_3a.to_csv('fig_3a.csv', index=False)


KeyError: "['fews_ipc_3', 'fews_ipc_6', 'fews_ipc_9', 'fews_ipc_12', 'fews_ipc_15', 'fews_ipc_18'] not in index"